In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define path to the dataset
root = 'content/drive/Shareddrives/EC523 Project/Dataset1_benchmark'


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define path to the dataset
root = 'content/drive/Shareddrives/EC523 Project/Dataset1_benchmark'

# Function to read tiff images
def read_tiff(path):
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img))
    return np.array(images)

# Load and store all images
images_allGenes = np.zeros((15, 25, 2006, 2005))
for i in range(1, 16):
    path = f'{root}/Gene{i}.tif'
    image = read_tiff(path)
    images_allGenes[i-1, :, :, :] = image

# Preprocess images
z1 = images_allGenes[:, 0, :, :]
transform = transforms.ToTensor()
z1 = transform(z1)
z1 = z1.permute(1, 2, 0)
z1 = torch.reshape(z1, (15, 2006 * 2005))
z1 = z1 / z1.max()

# Define autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(2006 * 2005, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2006 * 2005),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Train autoencoder
model = Autoencoder()
model.double()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

epochs = 10
encoded_list = []
decoded_list = []
loss_track = []

print('Start training')
for epoch in range(epochs):
    for batch in range(len(z1)):
        vec = z1[batch, :]
        encoded, decoded = model(vec)
        loss = criterion(decoded, vec)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch == epochs - 1:
            encoded_list.append(encoded.detach().numpy())
            decoded_list.append(decoded.detach().numpy())
    loss_track.append(loss.item())

# Save outputs
import pickle

with open('decoded', 'wb') as fp:
    pickle.dump(decoded_list, fp)

# Reshape decoded output
decoded_list = np.array(decoded_list)
decoded_vec = [decoded_list[:, i] for i in range(decoded_list.shape[1])]

# Perform k-means clustering on the decoded output
kmeans = KMeans(n_clusters=48)
kmeans_decoded = kmeans.fit(decoded_vec)
labels_decoded = kmeans_decoded.labels_
labels_decoded = labels_decoded.reshape((images_allGenes.shape[2],
images_allGenes.shape[3]))

# Visualize the clustered output
plt.imshow(labels_decoded, interpolation='nearest')
plt.savefig('./z1_decoded_48.jpg', dpi=300)
plt.show()

# Plot the loss curve
plt.plot(loss_track)
plt.savefig('./z1_loss_plot.jpg', dpi=300)
